In [ ]:
import satimagets as sits
import pandas as pd
import numpy as np
import re
import gdal
import matplotlib.pyplot as plt

In [ ]:
S2_PATH = "Y:\\Harmonisation_PS_S2\\m3sat_s2output\\"#"D:\\M3sat_S2output\\"
PS_PATH = "Y:\PS\Mosaic\PS_5m\\"
PS_OLD = "Y:\\Harmonisation_PS_S2\\output_S2_spatial\\"
PS_PATHre = "Y:\\\\Harmonisation_PS_S2\\\\m3sat_s2output\\\\"
PS_OLDre = "Y:\\\\Harmonisation_PS_S2\\\\output_S2_spatial\\\\"
locations = ["Izola", "Radenci", "Jesenice", "Kranj"]
DF_PATH = "C:\\Users\\mracic\\Univerza v Ljubljani\\Fetai, Bujar - satimagets-master\\"
dfs = pd.read_excel(DF_PATH+"Harmonisation_PS_S2.xlsx")

In [ ]:
# fix the path to S2 file
dfs["S2_path"] = dfs["S2_path"].apply(lambda x: re.sub(PS_OLDre,PS_PATHre,x))
# Add paths to masks
dfs["S2_mask_path"] = dfs["S2_path"].apply(lambda x: re.sub("m_S2","m_mask_S2",x))
dfs["PS_udm_path"] = dfs["PS_Path"].apply(lambda x: re.sub("analytic","udm",x))

In [ ]:
def open_im(im_file):
    try:
        return gdal.Open(im_file)
    except:
        print('Could not open:', im_file)
        sys.exit(1)

In [ ]:
usable_thres = 0.3
coeff_df = pd.DataFrame(columns=["PS_filename", "PS_Date", "S2_filename", "S2_Date", "band", "perc_valid", "m", "b"])
skip_list = ["I5_20181004_101f_analytic_mosaic.tif", "K5_20190506"] # K5_20190506_1040_udm_mosaic.tif 
skip = False 
for loc_name in locations:
    print(loc_name)
    sub_dfs = dfs[dfs["Location"]==loc_name]
    file_1_src = open_im(sub_dfs.iloc[0]["PS_Path"])
    file_2_src = open_im(sub_dfs.iloc[0]["S2_path"])
    # Find overlap
    im_bb, im_res, im_srs = sits.image_raster_overlap(file_1_src, file_2_src, 0)
    # Set resolution
    im_res = 100
    for index, row in sub_dfs.iterrows():
        skip = False
        file_1 = row["PS_Path"]
        file_1_mask = row["PS_udm_path"]
        file_2 = row["S2_path"]
        file_2_mask = row["S2_mask_path"]
        # check if file is on the skip list
        for x in skip_list:
            if x in file_1:
                skip = True
        # do the skip
        if skip:
            continue
        # Load images and masks
        file_1_src = open_im(file_1)
        file_2_src = open_im(file_2)
        file_1_mask = open_im(file_1_mask)
        file_2_mask = open_im(file_2_mask)
        # Resample images and masks
        im_1 = sits.image_raster_resample(file_1_src, im_bb, im_res, im_srs, 0)
        im_2 = sits.image_raster_resample(file_2_src, im_bb, im_res, im_srs, 0)
        im_1_m = sits.image_raster_resample(file_1_mask, im_bb, im_res, im_srs)
        im_2_m = sits.image_raster_resample(file_2_mask, im_bb, im_res, im_srs)
        # Merge masks
        mask = sits.mask_data(im_2_m, im_1_m, 0)
        mask = sits.mask_data(mask, im_2_m, 100)        
        # Mask images
        im_1 = sits.mask_data(im_1, mask, 100)
        im_2 = sits.mask_data(im_2, mask, 100)        
        # Add NDVI
        
        # Skip image if it is too cloudy or has only one value
        perc_valid = np.sum(~np.isnan(mask))/mask.size
        if perc_valid < usable_thres or len(np.unique(im_1[:,~np.isnan(mask)])) < 2:
            continue

        if im_1.shape[0] < 1:
            sits.image_scatterplot(im_1[[1,2,3],:,:]/10000, im_2[[1, 2, 3],:,:], im1_in_name="Planet", im2_in_name="Sentinel", sample=10000)
        for band in range(im_1.shape[0]):
            m, b = np.polyfit(im_1[band][~np.isnan(mask)], im_2[band][~np.isnan(mask)], 1)
            coeff_df.loc[len(coeff_df.index)] = np.append(row[["PS_filename", "PS_Date", "S2_filename", "S2_Date"]].values,[band, perc_valid, m, b])
        
        
        # Visualize correlation per band
        #sits.image_scatterplot(im_1[[1,2,3],:,:]/10000, im_2[[1, 2, 3],:,:], im1_in_name="Planet", im2_in_name="Sentinel", sample=10000)
        
    coeff_df.to_csv("csv/"+loc_name+"_coeff.csv")